In [74]:
import torch
from torch import nn
import pandas as pd
from utils import *
from dataset import CustomEmbeddingDataset
import torch.nn.functional as F

In [16]:
from sentence_transformers import SentenceTransformer, InputExample

device = 'cuda'
model_id = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_id).to(device)

In [17]:
model.encode(['hello', 'next'], convert_to_tensor=True).shape

torch.Size([2, 384])

In [25]:
ctrain = pd.read_csv('data/preprocessed/chaos_train.csv')

In [32]:
torch.tensor([[float(y) for y in string_of_list_to_list(x)] for x in ctrain['label_dist']]).shape

torch.Size([2490, 3])

In [37]:
model.encode(ctrain['hypothesis'], convert_to_tensor=True, show_progress_bar=True)[5, :].shape

Batches:   0%|          | 0/78 [00:00<?, ?it/s]

torch.Size([384])

In [40]:
train = CustomEmbeddingDataset(sentences_file='data/preprocessed/chaos_train.csv', encoder=model, device=device)

Batches:   0%|          | 0/78 [00:00<?, ?it/s]

Batches:   0%|          | 0/78 [00:00<?, ?it/s]

In [42]:
for idx, m in enumerate(model.modules()):
    print(idx, '->', m)

0 -> SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)
1 -> Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
2 -> BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            

In [58]:
loss = nn.CrossEntropyLoss()

In [68]:
logits = torch.tensor([[1.,2.,3.,4.]], requires_grad=True)
label = torch.tensor([[0.1,0.2,0.3,0.4]])

In [69]:
# inputt = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
output = loss(logits, label)
# output.backward()

In [70]:
output

tensor(1.4402, grad_fn=<DivBackward1>)

In [66]:
logits

tensor([[1, 2, 3, 4]])

In [71]:
class SoftCrossEntropyLoss(nn.Module):
    def __init__(self, weights):
      super().__init__()
      self.weights = weights

    def forward(self, y_hat, y):
      p = F.log_softmax(y_hat, 1)
      w_labels = self.weights*y
      loss = -(w_labels*p).sum() / (w_labels).sum()
      return loss

In [77]:
loss = SoftCrossEntropyLoss(torch.ones(1,4))

In [78]:
loss(logits, label)

tensor(1.4402, grad_fn=<DivBackward0>)

In [85]:
train[0]['label'].shape

torch.Size([3])